In [1]:
import os, sys, shutil
from datetime import *
from PIL import Image, ImageDraw, ImageFont
from operator import itemgetter
import numpy as np
from os import environ

In [2]:
output_file= environ.get("output_file","")
dataset_dir= environ.get("dataset_dir","")
tim= environ.get("tim","")
category = environ.get("category","")

input_dir = os.path.join(dataset_dir, tim, category)
videos = os.listdir(input_dir)

In [3]:
dates_of_video = []
for video in videos:
    name_of_video = os.path.basename(video)

    # str_date_of_video = 2016-06-20T18%3A46%3A22
    str_date_of_video = "T".join(os.path.splitext(name_of_video)[0].split("_")[-2:])

    # date_of_video = datetime(2016-06-20 18:46:22)
    date_of_video = datetime.strptime(str_date_of_video,"%Y-%m-%dT%H%%3A%M%%3A%S")

    #convert to argentinian date
    date_of_video -= timedelta(hours=3)
    
    dates_of_video.append((date_of_video, video))
    
dates_of_video = sorted(dates_of_video, key=itemgetter(0))

In [4]:
sorted_frames = []
for dat, video in dates_of_video:
    video_dir = os.path.join(input_dir, video)
    video_frames = os.listdir(video_dir)
    
    #[frame_id, frame_basename]
    video_frames_with_ids = [(int(os.path.splitext(name)[0].replace("_","")), name) for name in video_frames if name.endswith(".png")]
    video_frames_with_ids = sorted(video_frames_with_ids, key=itemgetter(0))
    for _,frame_basename in video_frames_with_ids:
        full_path = os.path.join(video_dir, frame_basename)
        
        img = Image.open(full_path)
        extrema = img.convert("L").getextrema()
        if abs(extrema[1]-extrema[0]) < 20:
            continue
        
        sorted_frames.append(full_path)
        
len(sorted_frames)

5324

In [5]:
with open(output_file, "w") as f:
    for frame_path in sorted_frames:
        f.write("%s\n"%frame_path)